In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [1]:
train = pd.read_csv('train.dat', sep='\n', header=None)
test = pd.read_csv('test.dat', sep='\n', header=None)

NameError: name 'pd' is not defined

In [ ]:
train_values = train.iloc[:,:].values
test_values = test.iloc[:,:].values

In [ ]:
docs, class_list, test_docs = [], [], []

for i in train_values:
    class_list.append(i[0][:1])
    docs.append(i[0][2:])
for i in test_values:
    test_docs.append(i[0])
    
print("train data length - ",len(docs))
print("test data length - ",len(test_docs))

In [3]:
class_list1,class_list2,class_list3,class_list4,class_list5 = 0,0,0,0,0
test_class_list = [int(i) for i in class_list]
for clas in test_class_list:
    if clas==1:
        class_list1+=1
        continue
    if clas==2:
        class_list2+=1
        continue
    if clas==3:
        class_list3+=1
        continue
    if clas==4:
        class_list4+=1
        continue
    if clas==5:
        class_list5+=1
test_class_list = np.asarray(test_class_list)
print(type(docs))
print(type(test_class_list))
docs = [i.split() for i in docs]
test_docs = [j.split() for j in test_docs]

NameError: name 'class_list' is not defined

In [4]:
def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]
docs = filterLen(docs, 4)
test_docs = filterLen(test_docs, 4)
docs1 = []
for i in docs:
    doc1 = []
    for j in i:
        docc = ''.join(c for c in j if c.isalnum())
        doc1.append(docc.lower())
    docs1.append(doc1)
docs = docs1
docs1 = []
for i in test_docs:
    doc1 = []
    for j in i:
        docc = ''.join(c for c in j if c.isalnum())
        doc1.append(docc.lower())
    docs1.append(doc1)
test_docs = docs1
print(len(docs[0]), docs[0][:20])
print(len(test_docs[0]), test_docs[0][:20])

NameError: name 'docs' is not defined

In [ ]:
def cmer(name, c=3):
    r""" Given a name and parameter c, return the vector of c-mers associated with the name
    """
    name = name.lower()
    if len(name) < c:
        return [name]
    v = []
    for i in range(len(name)-c+1):
        v.append(name[i:(i+c)])
    return v

In [7]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat, idx

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

train_mat, idx = build_matrix(docs)
csr_info(train_mat,non_empy=True)



/var/folders/sn/jv8q19ts7p1cd4qlt53d6w800000gn/T/ipykernel_8807/674298888.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ind = np.zeros(nnz, dtype=np.int)
/var/folders/sn/jv8q19ts7p1cd4qlt53d6w800000gn/T/ipykernel_8807/674298888.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

 [nrows 14438 (14438 non-empty), ncols 50984 (50984 non-empty), nnz 1132488]


In [8]:
def build_test_matrix(docs,idx):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)

    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            temp = idx.get(k,-1)
            if temp != -1:
                ind[j+n] = temp
                val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat
test_mat = build_test_matrix(test_docs,idx)
csr_info(test_mat,non_empy=True)

def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum   
    if copy is True:
        return mat

/var/folders/sn/jv8q19ts7p1cd4qlt53d6w800000gn/T/ipykernel_8807/4160708262.py:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ind = np.zeros(nnz, dtype=np.int)
/var/folders/sn/jv8q19ts7p1cd4qlt53d6w800000gn/T/ipykernel_8807/4160708262.py:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current

 [nrows 14442 (14442 non-empty), ncols 50984 (28623 non-empty), nnz 1153688]


In [9]:
train_mat1 = csr_idf(train_mat, copy=True)
train_mat2 = csr_l2normalize(train_mat1, copy=True)
print("train_mat2:", train_mat2[15,:20].todense(),"\n")
print(train_mat2.shape)

train_mat2: [[0.         0.         0.         0.         0.         0.00717046
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.03464385
  0.02330707 0.        ]] 

(14438, 50984)


In [11]:
test_mat1 = csr_idf(test_mat, copy=True)
test_mat2 = csr_l2normalize(test_mat1, copy=True)
print("test_mat2:", test_mat2[15,:20].todense(),"\n")
print(test_mat2.shape)

test_mat2: [[0.         0.         0.         0.         0.         0.01340504
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.07495871
  0.00842371 0.        ]] 

(14442, 50984)
